In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
clear_df = pd.read_csv('../readyDatasets/preprocessed_dataframe.csv')

In [3]:
X = clear_df.drop(columns='УСЬОГО')
y = clear_df['УСЬОГО']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [4]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

# Регуляризація та відсіювання маловпливових ознак
Навчимо модель з L1-регуляризацією, щоб вирішити проблему мультиколінеарності. Коефіцієнти при сильно скорельованих ознаках повинні бути близькі до нуля.

In [5]:
alphas = np.linspace(0.001, 5, 500)
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=50000)
lasso_cv.fit(X_train, y_train)

print('Найменш важливі фічі:', X.columns[np.where(lasso_cv.coef_ == 0)])
best = np.where(lasso_cv.coef_ != 0)

Найменш важливі фічі: Index(['Середній пріоритет рекомендованих', 'Рекомендовано за співбесідою',
       'Регіональний коефіцієнт'],
      dtype='object')


In [6]:
lasso = Lasso(alpha=lasso_cv.alpha_)
lasso.fit(X_train, y_train)

C:\Users\natan.chachko\PycharmProjects\pythonProject\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.065e+06, tolerance: 2.147e+03
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Lasso(alpha=0.04107214428857715)

In [7]:
alphas = np.linspace(0.01, 1000, 2000)
ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(X_train[X.columns[best]], y_train)
pd.DataFrame(ridge_cv.coef_, X.columns[best], columns=["coef"]).sort_values(by="coef", ascending=False)

,coef
Середній пріорітет рекомендованих на загальних підставах,1.151872
квота-1,1.075106
form,0.613948
Макс. обсяг держзамовлення,0.530231
Макс. Бал (на загальних підставах),0.477355
Допущено до конкурсу,0.260255
СЕР,0.200119
Рік,0.178662
uni_code,0.176887
spec_full,0.112935


In [8]:
ridge = Ridge(alpha=ridge_cv.alpha_)
ridge.fit(X_train[X.columns[best]], y_train)

Ridge(alpha=262.13844422211105)

In [10]:
y_pred = linreg.predict(X_test)
y_pred_lasso = lasso.predict(X_test)
y_pred_ridge = ridge.predict(X_test[X.columns[best]])

print("Linear regr:\t MSE =", mean_squared_error(y_test, y_pred), '\t R2 =', r2_score(y_test, y_pred))
print("Lasso:\t\t\t MSE =", mean_squared_error(y_test, y_pred_lasso), '\t R2 =', r2_score(y_test, y_pred_lasso))
print("Ridge:\t\t\t MSE =", mean_squared_error(y_test, y_pred_ridge), '\t R2 =', r2_score(y_test, y_pred_ridge))

Linear regr:	 MSE = 275.79128785392237 	 R2 = 0.8222631534996667
Lasso:			 MSE = 275.30546391916175 	 R2 = 0.822576248285186
Ridge:			 MSE = 275.97643746072777 	 R2 = 0.8221438317201412
